In [1]:
!pip -qqq install pip --progress-bar off

In [2]:
!pip -qqq install 'crewai[tools]'==0.28.8 --progress-bar off

!pip -qqq install langchain-groq==0.1.3 --progress-bar off

!pip install exa-py

In [4]:
!pip uninstall -y numpy pandas
!pip install numpy==1.26.4 pandas==2.2.2


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached pandas-2.2.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
Using cached pandas-2.2.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatibl

In [ ]:
import pandas as pd
from exa_py import Exa
import requests
import os


os.environ["GROQ_API_KEY"]=""
exa = Exa(api_key="")


In [9]:
def get_cryptocurrency_results(query):
  result = exa.search_and_contents(query,summary=True)
  if result.results:
    news_list=[]
    for item in result.results:
      news_item = {

                "title": item.title if hasattr(item, "title") else "No Title",

                "url": item.url if hasattr(item, "url") else "#",  # URL and ID are the same

                "id": item.id if hasattr(item, "id") else "#",  # ID is same as URL

                "score": item.score if hasattr(item, "score") else "No Score",

                "published_date": item.published_date if hasattr(item, "published_date") else "Unknown Date",

                "author": item.author if hasattr(item, "author") else "Unknown Author",

                "image": item.image if hasattr(item, "image") else "No Image",

                "favicon": item.favicon if hasattr(item, "favicon") else "No Favicon",

                "summary": item.summary if hasattr(item, "summary") else "No Summary",

                "highlights": item.highlights if hasattr(item, "highlights") else "No Highlights",

                "highlight_scores": item.highlight_scores if hasattr(item, "highlight_scores") else "No Highlight Scores",

            }
      news_list.append(news_item)

    return news_list



get_cryptocurrency_results("Latest Cryptocurrency news about Bitcoin")

[{'title': 'BITCOIN BEARISH TARGETS HIT (this is coming next ...',
  'url': 'https://www.youtube.com/watch?v=ltERqddqnMw',
  'id': 'https://www.youtube.com/watch?v=ltERqddqnMw',
  'score': None,
  'published_date': '2026-01-20T20:00:17.703Z',
  'author': 'Crypto World',
  'image': 'https://i.ytimg.com/vi/ltERqddqnMw/hqdefault.jpg',
  'favicon': None,
  'summary': 'The video titled "BITCOIN BEARISH TARGETS HIT (this is coming next)!!!" provides an analysis of the current state of Bitcoin and other cryptocurrencies. It discusses recent price movements, bearish targets for Bitcoin, and includes price analyses for Ethereum, Solana, XRP, and Chainlink. Key timestamps highlight Bitcoin\'s price analysis starting at 2:38, with additional insights into market trends and dominance. The content is aimed at educating viewers about potential market shifts and trading strategies, although it emphasizes that the information is not financial advice. For the latest updates on Bitcoin and other cryptoc

In [ ]:
def get_daily_closing_prices(ticker)-> pd.DataFrame:
  api_key=""
  url = f"https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol={ticker}&market=USD&apikey={api_key}"

  response = requests.get(url)
  data = response.json()
  price_data = data["Time Series (Digital Currency Daily)"]
  daily_closing_prices ={
     date: prices["4. close"] for (date,prices) in price_data.items()
  }

  df = pd.DataFrame.from_dict(daily_closing_prices,orient="index",columns=["price"])
  df.index = pd.to_datetime(df.index)
  df.price = pd.to_numeric(df.price)
  return df


closing_prices_df=get_daily_closing_prices("BTC")

In [21]:
from crewai_tools import tool
from langchain.agents import load_tools

@tool("search_tool")
def search_tool(query: str) -> str:
  """Search for the latest news and retrieve the summary about given query using Exa """

  result = exa.search_and_contents(query,summary=True)
  if result.results:
    news_list=[]
    for item in result.results:
      news_item = {

                "title": item.title if hasattr(item, "title") else "No Title",

                "url": item.url if hasattr(item, "url") else "#",  # URL and ID are the same

                "id": item.id if hasattr(item, "id") else "#",  # ID is same as URL

                "score": item.score if hasattr(item, "score") else "No Score",

                "published_date": item.published_date if hasattr(item, "published_date") else "Unknown Date",

                "author": item.author if hasattr(item, "author") else "Unknown Author",

                "image": item.image if hasattr(item, "image") else "No Image",

                "favicon": item.favicon if hasattr(item, "favicon") else "No Favicon",

                "summary": item.summary if hasattr(item, "summary") else "No Summary",

                "highlights": item.highlights if hasattr(item, "highlights") else "No Highlights",

                "highlight_scores": item.highlight_scores if hasattr(item, "highlight_scores") else "No Highlight Scores",

            }
      news_list.append(news_item)
    output=[]

    for news_item in news_list:
      output.append(f"Title: {news_item['title']} Url: {news_item['url']} Summary: {news_item['summary']}")

    return ".\n".join(output)
  else:
    return "No results found"


In [22]:
@tool("price tool")
def Cryptocurrency_price_tool(ticker_symbol: str) -> str:
  """Search the price history for the given cryptocurrency ticker symbol for the previous 60 days """
  price_df = get_daily_closing_prices(ticker_symbol)

  text_output = []

  for date, row in price_df.head(60).iterrows():

      text_output.append(f"{date.strftime('%Y-%m-%d')} - {row['price']:.2f}")

  return "\n".join(text_output)

In [23]:
@tool("search tool")
def cryptocurrency_news_tool(ticker_symbol: str) -> str:
  """Given the news for the given cryptocurrency ticker symbol """

  return search_tool.run(ticker_symbol + "cryptocurrency")

In [25]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

llm = ChatGroq(temperature=0,model_name="llama-3.3-70b-versatile")

system_message = "You are an experienced Machine Learning & AI Engineer."

human_message = "How to increase inference speed for a 7B LLM?"

prompt = ChatPromptTemplate.from_messages([("system", system_message), ("human", human_message)])

chain = prompt | llm

response = chain.invoke({"text": human_message})

print("\nLLM Response:\n", response.content)


LLM Response:
 Increasing inference speed for a large language model (LLM) like a 7B model requires a combination of techniques that optimize the model's architecture, computational resources, and deployment strategy. Here are some methods to improve inference speed:

### 1. **Model Pruning and Quantization**

*   **Model Pruning**: Remove redundant or unnecessary weights and connections in the model to reduce computational requirements. This can be done using techniques like iterative magnitude pruning or movement pruning.
*   **Quantization**: Represent model weights and activations using lower-precision data types (e.g., int8 instead of float32) to reduce memory usage and increase computation speed.

### 2. **Knowledge Distillation**

*   Train a smaller "student" model to mimic the behavior of the larger "teacher" model (the 7B LLM). The student model can be designed to be more efficient while maintaining similar performance.

### 3. **Efficient Attention Mechanisms**

*   **Spars

In [29]:
from crewai import Agent, Task,Crew, Process

customer_communicator = Agent(
    role="Senior cryptocurrency customer communicator",
    goal = "Find which currency is user intrested in",
    backstory ="You are highly experienced in communication about cryptocurriences and blockchain technology with customers and their research needs",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    max_iter=5,
    memory=True
)

news_analyst = Agent(
    role="Senior Cryptocurrency News Analyst",
    goal="Get news for the given cryptocurrency. Write 1 paragraph analysis of the market and prediction - up down and neutral",
    backstory="You are an expert analyst of trends in cryptocurrency news.  You have a complete understanding of macroeconomic factors, but you specialize into analyzing news.",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=[cryptocurrency_news_tool]
)

price_analyst = Agent(
    role="Senior Cryptocurrency Price Analyst",
    goal="Get historical prices for the given cryptocurrency. Write 1 paragraph analysis of the market and prediction - up down and neutral",
    backstory="You are an expert analyst of trends based on cryptocurrency historical prices.  You have a complete understanding of macroeconomic factors, but you specialize into analyzing historical prices.",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=[Cryptocurrency_price_tool]
)

writer = Agent(
    role="Cryptocurrency Report Writer",
    goal="""Write 1 paragraph report of the cryptocurrency market.""",
    backstory="""

    You're widely accepted as the best cryptocurrency analyst that

    understands the market and have tracked every asset for more than 10 years. Your trends

    analysis are always extremely accurate.

    You're also master level analyst in the traditional markets and have deep understanding

    of human psychology. You understand macro factors and combine those multiple

    theories - e.g. cycle theory. You're able to hold multiple opinions when analysing anything.

    You understand news and historical prices, but you look at those with a

    healthy dose of skepticism. You also consider the source of news articles.

    Your most well developed talent is providing clear and concise summarization

    that explains very complex market topics in simple to understand terms.

    Some of your writing techniques include:

    - Creating a bullet list (executive summary) of the most important points

    - Distill complex analyses to their most important parts

    You writing transforms even dry and most technical texts into

    a pleasant and interesting read.""",

    llm=llm,

    verbose=True,

    max_iter=5,

    memory=True,

    allow_delegation=False,

)


In [37]:
from datetime import datetime
get_cryptocurrency= Task(
    description="Ask which cryptocurrency the customer is intrested in ",
    expected_output = "Crypto currency symbol that the human wants you to research ex: BTC, ETH..",
    agent=customer_communicator
)

get_news_analysis= Task(
    description     = f"Use the search tool to get the current news about the requested cryptocurrency. Current date is {datetime.now()} Compose the result in a helpful report",
    expected_output = "Create 1 paragraph report for the cryptocurrency news along with  prediction of the future trend",
    agent=news_analyst,
    context=[get_cryptocurrency]
)

get_historical_price_analysis= Task(
    description     = f"Use the search tool to get the historical prices about the requested cryptocurrency. Current date is {datetime.now()} Compose the result in a helpful report",
    expected_output = "Create 1 paragraph summary for the cryptocurrency prices along with  prediction of the future trend",
    agent=news_analyst,
    context=[get_cryptocurrency]
)

writer_report = Task(
     description=f"""Use the reports from the news analyst and the price analyst to
    create a report that summarizes the cryptocurrency""",
     expected_output="""1 paragraph report that summarizes the market and
    predicts the future prices (trend) for the cryptocurrency""",
     agent=writer,

     context=[get_news_analysis, get_historical_price_analysis],
)


In [38]:
crew = Crew(
    agents=[customer_communicator,price_analyst,news_analyst,writer],
     tasks=[get_cryptocurrency, get_news_analysis, get_historical_price_analysis, writer_report],
    verbose=2,
    process= Process.sequential,
    full_output=True,
    share_crew=False,
    manager_llm=llm,
    max_iter=5
)

results = crew.kickoff()

 [DEBUG]: == Working Agent: Senior cryptocurrency customer communicator
 [INFO]: == Starting Task: Ask which cryptocurrency the customer is intrested in 


> Entering new CrewAgentExecutor chain...
To determine which cryptocurrency the customer is interested in, I will need to engage in a conversation and ask a series of questions. However, given the constraints of this interaction, I will proceed with a direct approach to fulfill the task requirements.

To identify the cryptocurrency of interest, I consider the most popular and widely recognized cryptocurrencies that often spark curiosity and investment interest. These include but are not limited to Bitcoin (BTC), Ethereum (ETH), Ripple (XRP), and Litecoin (LTC). Each of these cryptocurrencies has unique features, use cases, and market dynamics that might appeal to different investors or enthusiasts.

Given the broad range of cryptocurrencies available, each with its own set of characteristics, use cases, and potential for growth or i